<a href="https://colab.research.google.com/github/3lueLightning/continente_ec2_scraper/blob/master/web_parser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
products_html_fn = 'products_html.pkl'
categories_fn = 'product_categories.csv'
base_dir = "/content/gdrive/My Drive/scraping/" 

In [0]:
from google.colab import drive
import os

drive.mount('/content/gdrive', force_remount=True)
os.makedirs(base_dir, exist_ok=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import pickle

def prod_unpickler(fn):
  products = {}
  with open(fn, 'rb') as load_file:
    while True:
      try:
        new_prods = pickle.load(load_file)
      except EOFError:
        break
      products.update(new_prods)
  return products

products_html = prod_unpickler(base_dir + products_html_fn)

In [0]:
#products_html = products_html_store.copy()

In [0]:
import pandas as pd

categories = pd.read_csv( base_dir + categories_fn)
categories

,category,category_lvl1
0,Bio e Saudável,Vegan e Vegetariano
1,Bio e Saudável,Sem Glúten
2,Bio e Saudável,Sem Lactose
3,Bio e Saudável,Biológicos
4,Bio e Saudável,Mercearia
...,...,...
133,Festa,Bebidas
134,Festa,Acessórios de Festa
135,Festa,Prendas
136,Festa,Aperitivos e Sobremesas


In [0]:
def price_parser(raw_price):
  price, unit = raw_price.split()[1:]
  price = float(price.replace(',', '.'))
  return price, unit

In [0]:
from bs4 import BeautifulSoup as soup

def extract_product(container):
  # product description
  success = True
  product_info ={}
  description_html = container.findChild("div",{"class":"containerDescription"})
  try:
    product_info['product'] = description_html.div.a["title"]
  except AttributeError:
    success = False
  try:
    product_info['brand'] = description_html.find("div",{"class":"type"}).text
  except AttributeError:
    pass
  try:
    details = description_html.find("div",{"class":"subTitle"}).text
  except AttributeError:
    pass
  else:
    product_info['details'] = details.strip()
  # product price
  price_html = container.findChild("div",{"class":"containerPrice"})
  try:
    price = price_html.find("div",{"class":"priceFirstRow"}).text
  except AttributeError:
    success = False
  else:
    try:
      product_info['price'], product_info['price_units'] = price_parser(price)
    except:
      success = False
  try:
    price_weight = price_html.find("div",{"class":"priceSecondRow"}).text
  except AttributeError:
    try:
      product_info['price_weight'], product_info['price_weight_units'] = price_parser(price_weight)
    except:
      success = False
  discount_html = container.findChild("div",{"class": "discountMessage priceWas"})
  try:
    product_info['price_pack_no_discount'] = discount_html.findChild("div",{"class": "priceFirstRow"}).text
  except AttributeError:
    pass
  try: 
    product_info['price_weight_no_discount'] = discount_html.findChild("div",{"class": "priceSecondRow"}).text
  except AttributeError:
    pass
  try:
    product_info['discount_text'] = discount_html.findChild("span",{"class": "iconDiscountText"}).text
  except AttributeError:
    pass
  prod_links = container.findChild('div', {'class': 'image'})
  try:
    product_info['image_url'] = prod_links.a.img['src']
  except AttributeError:
    pass
  try:
    product_info['url'] = prod_links.a['href']
  except AttributeError:
    pass
  return success, product_info

In [0]:
container = soup(products_html[0][0])

In [0]:
from collections import defaultdict

prods_not_parsed = defaultdict(list)
prods_parsed = []
for category_id, containers in products_html.items():
  i = 0
  for container in containers:
    success, prod = extract_product( soup(container))
    if success:
      prod['category_id'] = category_id
      prods_parsed.append(prod)
    else:
      prods_not_parsed[category_id].append(i)
    i += 1

In [0]:
prods_parsed[0]

{'brand': 'Continente',
 'category_id': 0,
 'details': 'emb. 300 gr',
 'image_url': 'https://media.continente.pt/Sonae.eGlobal.Presentation.Web.Media/media.axd?resourceSearchType=2&resource=ProductId=6739066(eCsf$RetekProductCatalog$MegastoreContinenteOnline$Continente)&siteId=1&channelId=1&width=150&height=150&defaultOptions=1',
 'price': 1.99,
 'price_units': '/un',
 'product': 'Couscous Quinoa e Vegetais',
 'url': 'https://www.continente.pt/stores/continente/pt-pt/public/Pages/ProductDetail.aspx?ProductId=6739066(eCsf_RetekProductCatalog_MegastoreContinenteOnline_Continente)&refiner=%23%2F%3Fpl%3D80%26page%3D1'}

In [0]:
with open(base_dir + 'parsed_products.pkl', 'wb') as save_file:
  pickle.dump( prods_parsed, save_file)